In [2]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import textblob
from textblob import TextBlob
import pandas as pd 

In [3]:
def properdatetimestamp(year,month,date):
    yearval = str(year)
    monthval = str(month)
    dayval = str(date)
    if len(monthval) < 2:
        monthval = "0" + monthval
    if len(dayval) < 2:
        dayval = "0" + dayval
    dateval = dayval + monthval + yearval
    return dateval

In [6]:
bitcoindata = open('bitcoin.csv', 'w')
bitcoindata.write("date" +  ',' + "price") 
bitcoindata.write('\n')
url = "https://api.coindesk.com/v1/bpi/historical/close.json?start=2018-03-15&end=2019-03-15"
response = requests.get(url)
if(response.status_code == 200):
    bitcoinval = json.loads(response.text)
    bitcoindic = bitcoinval['bpi']
    for date, price in bitcoindic.items(): 
        year,month,day = date.split("-");
        final_date  = properdatetimestamp(year,month,day);
        bitcoindata.write(final_date +  ',' + str(price)) 
        bitcoindata.write('\n')
    bitcoindata.close()
else:
    print("Error status: "+str(response.status_code))

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20180315&end=20190315'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

priceDiv = soup.find('div', attrs={'class':'table-responsive'})
rows = priceDiv.find_all('tr')

data = []
i = 0;

for row in rows:
    tmp = []
    tds = row.findChildren()

    for td in tds:
        tmp.append(td.text)

    if(i > 0):
        tmp[0] = tmp[0].replace(',','')
        tmp[5] = tmp[5].replace(',','')
        tmp[6] = tmp[6].replace(',','')
        data.append({'date':datetime.strptime(tmp[0], '%b %d %Y'),
                     'open':float(tmp[1]),
                     'high':float(tmp[2]),
                     'low':float(tmp[3]),
                     'close':float(tmp[4]),
                     'volume':float(tmp[5]),
                     'mcap':float(tmp[6])})

    i = i + 1;

df = pd.DataFrame(data)

In [3]:
df.head()

,close,date,high,low,mcap,open,volume
0,3960.91,2019-03-15,3968.54,3914.02,6.967500e+10,3926.66,9.394211e+09
1,3924.37,2019-03-14,3946.50,3901.30,6.902470e+10,3905.58,1.048079e+10
2,3906.72,2019-03-13,3926.60,3891.90,6.870670e+10,3913.05,9.469185e+09
3,3909.16,2019-03-12,3926.89,3863.56,6.874300e+10,3903.76,9.809887e+09
4,3905.23,2019-03-11,3966.38,3889.24,6.866693e+10,3953.74,1.012590e+10


In [4]:
bitcoincsv = df.to_csv ("bitcoin.csv", index = None, header=True)